In [5]:
import torch
import argparse

In [6]:
args = {
    'config': 'configs/scalar.txt',
    'expname': 'scalar_test1',
    'basedir': './log',
    'datadir': './data/ScalarReal',
    'net_model': 'siren',
    'netdepth': 8,
    'netwidth': 256,
    'netdepth_fine': 8,
    'netwidth_fine': 256,
    'N_rand': 1024,
    'lrate': 0.0005,
    'lrate_decay': 500,
    'chunk': 32768,
    'netchunk': 65536,
    'no_batching': True,
    'no_reload': False,
    'ft_path': None,
    'fix_seed': 42,
    'fading_layers': 50000,
    'tempo_delay': 0,
    'vel_delay': 10000,
    'N_iter': 600000,
    'train_warp': True,
    'bbox_min': '0.05',
    'bbox_max': '0.9',
    'vgg_strides': 4,
    'ghostW': 0.07,
    'vggW': 0.01,
    'overlayW': -0.0,
    'd2vW': 2.0,
    'nseW': 0.001,
    'vol_output_only': False,
    'vol_output_W': 128,
    'render_only': False,
    'render_test': False,
    'N_samples': 64,
    'N_importance': 64,
    'perturb': 1.0,
    'use_viewdirs': False,
    'i_embed': -1,
    'multires': 10,
    'multires_views': 4,
    'raw_noise_std': 0.0,
    'render_factor': 0,
    'precrop_iters': 1000,
    'precrop_frac': 0.5,
    'dataset_type': 'pinf_data',
    'testskip': 20,
    'shape': 'greek',
    'white_bkgd': True,
    'half_res': 'half',
    'factor': 8,
    'no_ndc': False,
    'lindisp': False,
    'spherify': False,
    'llffhold': 8,
    'i_print': 400,
    'i_img': 2000,
    'i_weights': 25000,
    'i_testset': 50000,
    'i_video': 50000
}
args = argparse.Namespace(**args)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.set_default_device('cuda')
torch.set_default_dtype(torch.float32)